# _Scala Fundamentals_
# 5. Classes, Traits, and Objects

_This module was originally developed by Brian Clapper._

%md

## Introduction

In this lesson, we're going to discusses Scala classes, Scala traits (which are analagous to Java interfaces, but more powerful), and Scala singleton objects.

If you know classes in Java, you'll find Scala classes to be refreshingly concise and easy to read.


## Classes

Let's do classes first.

### Key points

- Scala automatically generates getters and setters. You don't have to code them.
- You can replace a field with a custom gettor or setter, without changing how callers access the 
  field. This is the [Uniform Access Principle](http://docs.scala-lang.org/glossary/#uniform-access-principle).
- You can use `@BeanProperty` if you need JavaBeans setters and getters.
- Every class has a primary constructor that is "interwoven" with the class definition.
- Auxiliary constructors are optional and are named `this`.

### Simple Classes and Parameterless Methods

Scala classes look a lot like their counterparts in Java, C#, C++, Python or Ruby:

In [ ]:
class Counter {
  private var value = 0 // You MUST initialize the field.
  
  // Methods are public by default.
  def increment(): Unit = {
    value += 1
  }
  
  def current() = value
}

Methods are just functions inside the class and, like, functions, are created with the `def` keyword.

### Some Differences from Java

* `public` is the default access modifier. Classes, fields, and methods are public, by default.
* A source file can contain multiple classes.
* There's no requirement that the class name match the source file name, as there is in Java.

### Using our `Counter` class

To use this class, you construct objects and invoke methods the usual way.

In [ ]:
val counter1 = new Counter
val counter2 = new Counter() // equivalent

counter1.increment()

In [ ]:
println(s"counter1=${counter1.current()}")
println(s"counter2=${counter2.current()}")

You can call parameterless methods, like `current()`, with or without parentheses. The following two calls are equivalent:

In [ ]:
println(counter1.current())
println(counter1.current)

![?](http://i.imgur.com/Guv4TBn.png) **Which form should you use?**

It is considered good style to use `()` for a mutator method (a method that changes object state or performs I/O) and to drop the `()` for an accessor method.

That's what we did in our example:

In [ ]:
counter1.increment() // Mutates state: Use ()
counter1.current     // Accesses value without changing state.

You can enforce the accessor style by declaring `current` without `()`:

In [ ]:
class Counter {
  private var value = 0

  def increment(): Unit = {
    value += 1
  }
  
  def current = value
}

### Properties (Fields) with getters and setters

When writing Java classes, we tend to avoid public fields. For instance, the following is considered to be bad Java code:

```java
public class Person { // This is Java, not Scala
    public int age;
    public String name;
}
```

The problem with this approach is its inflexibility. If you publish this class, callers will access `age` as follows:

```java
Person fred = new Person();
fred.name = "Fred";
fred.age = 21;
```

But, what if we want to ensure that a caller does not make a person younger? To do that in Java, we have to change the public interface of the class
to add a setter.

For this reason, when programming Java, we defensively write setters and getters:

```java
public class Person {
    private int age;
    private String name;
    
    public int getAge() { return age; }
    
    public void setAge(int newAge) {
        // Now we can add some checks.
        if (newAge > this.age)
            this.age = newAge;
    }
    
    public String getName() { return name; }
    
    public void setName(String newName) { this.name = newName; }
}
```

A getter/setter pair, such as `getAge()` and `setAge()`, is often called a _property_. So, `Person` has two properties: `age` and `name`.

### The problem with Java's approach

The Java approach is highly verbose. 

- When we write Java classes, we defensively write getters and setters for _every_ public field.
- Most of the getters only return their corresponding values.
- Most of the setters only set their values.

We have to write (and read) a bunch of extra boilerplate code that doesn't actually solve the problem the class is intended to solve.

### Other languages don't do what Java does

Languages like Python, Ruby, C# and Scala:

- allow you to access a field as if it were public `fred.age`
- provide the ability to intercept get and set operations behind the scenes

- In Python and C#:
    - You start out with public fields.
    - If you need a setter or getter, you can later add a property declaration.
- In Ruby, you can mark a field as an attribute, and it will generate a setter and/or getter for
  you, under the covers. 
- Scala automatically generates default getters and setters.

In all cases, _you can continue to access the field as if it were public._

### An example

In [ ]:
class Person {
  var age = 0
  var name = ""
}

During compilation, Scala converts `age` and `name` to private fields and generates setters and getters. For instance, here's
the result of running `javap -p` on the compiled `Person` class:

<pre>
public class Person {
  private int age;
  private java.lang.String name;
  public int age();
  public void age_$eq(int);
  public java.lang.String name();
  public void name_$eq(java.lang.String);
  public Person();
}
</pre>

#### What happened?

* Scala converted `age` to a private field.
* It generated an `age()` getter, which we can just invoke without the parentheses.
* It generated an `age_=()` setter, which we can just invoke with the assignment operator.
* It did the same for the `name` field.

In [ ]:
val person = new Person
person.name = "Fred" // calls person.name_=()
person.age = 21      // calls person.age_=()

In [ ]:
println(s"${person.name} is ${person.age} years old.")

Note that assignment to `name` is just _syntactic sugar_ for calling the `name_=` method:

In [ ]:
person.name_=("Annabelle")
println(s"${person.name} is ${person.age} years old.")

To add a check to ensure that a caller can't make a person younger

- Define the setter explicitly.
- Doing so prevents Scala from defining a default one.

In [ ]:
class Person {
  private var theAge = 0
  var name = ""
  
  def age_=(newAge: Int): Unit = {
    if (newAge > this.theAge)
      this.theAge = newAge
  }
  
  def age = this.theAge
}

In [ ]:
val person = new Person
person.name = "Maria"
person.age = 43
person.age = 10
println(person.age)


#### Lack of boilerplate

Compare our Scala `Person` class with its Java equivalent. 

- In Scala, we don't have to define every getter and setter _just in case_. 
- Instead, we allow Scala to generate getters and setters for us, overriding only the ones we need.

This approach leads to much more compact and readable code.

### Uniform Access Principle

<div style="border: 1px solid #ddd; border-radius: 10px 10px 10px 10px; padding: 10px; background-color: #f7edcd; width: 40%; margin: 20px">
Bertrand Meyer, the inventor of the influential Eiffel language, formulated the Uniform Access Principle. It states:

<p style="margin-left: 1em; margin-right: 1em; margin-top: 0.5em">
All services offered by a module should be available through a uniform notation, which does not betray whether they are implemented through storage or through computation.
In Scala, the caller of <tt>fred.age</tt> doesn’t know whether age is implemented through a field or a method. (Of course, in the JVM, the service is always implemented through a method, either synthesized or programmer-supplied.)
</p>

</div>

### Properties with only getters

If you define a field as a `val`, Scala will generate only a getter, since a `val` cannot be changed.

In [ ]:
class Message {
  val timestamp = new java.util.Date
}

In [ ]:
// This works:
val m = new Message
println(m.timestamp)

// This won't compile.
//m.timestamp = new java.util.Date

### Properties with only setters

You cannot define a set-only field in Scala. If you try to do, your class will compile, but you'll get a compilation error trying to call the setter.

In [ ]:
class DoesNotWork {
  private var someValue: Int = 0
  
  def value_=(newValue: Int): Unit = {
    someValue = newValue
  }
}

In [ ]:
val dnw = new DoesNotWork
dnw.value = 30

### Summary: Scala setters and getters

You have four choices for implementing properties:

1. `var foo`: Scala generates a getter (`foo`) and a setter (`foo_=()`).
2. `val foo`: Scala generates a getter.
3. You can define your own getter and setter by providing the `foo` and `foo_=()` methods.
4. You can define your own getter by providing just the `foo` method.

### Exercise 5.1: Implement a simple class

Implement the following class, and run the cell after it to test that you got it right.

**Requirements**

1. The `deposit()` method should not permit a negative deposit. (That would be a withdrawal.)
2. The `withdrawal()` method should not allow an overdraft. It should return the smaller of the current balance and the desired amount. 
   That is, if someone attempts to withdraw 20.00, and the balance is only 15.00, `withdraw()` should zero the balance and return 15.00.
   It should _not_ throw an exception.
3. `balance` should return the current balance.
4. The balance should be initialized to 0.

**Notes**: 

1. In Scala, you can use `BigDecimal` the way you use any other numeric type. All the normal operators work fine. You can
   create a BigDecimal from an integer or floating point number easily: `BigDecimal(120)`.
2. We're using _mutable_ instance data here, which isn't considered good practice in Scala. 
   Don't worry about that for now. At the moment, we're just trying to get our heads around the syntax.
   
**Question**: Why are we using `BigDecimal`, rather than, say, `Double`?

In [ ]:
class BankAccount {
  def deposit(amount: BigDecimal): Unit = ???
  
  def withdraw(amount: BigDecimal): BigDecimal = ???
  
  def balance = ???
}

In [ ]:
// ANSWER
class BankAccount {
  private var currentBalance = BigDecimal(0)
  
  def deposit(amount: BigDecimal): Unit = {
    if (amount >= 0) currentBalance += amount
  }
  
  def withdraw(amount: BigDecimal): BigDecimal = {
    val actualAmount = if (amount > currentBalance)
      currentBalance
    else
      amount
    
    currentBalance -= actualAmount
    actualAmount
  }
  
  def balance = currentBalance
}

In [ ]:
// Run this cell to test your solution.

val acct = new BankAccount
assert(acct.balance == 0, "Balance not initialized to 0.")
acct.deposit(-100)
assert(acct.balance == 0, "Balance incorrectly update by negative deposit.")
acct.deposit(100)
assert(acct.balance == 100, "Deposit of 100 didn't update balance properly.")
assert(acct.withdraw(50) == 50, "Withdrawal returned wrong amount with sufficient balance.")
assert(acct.balance == 50, "Balance incorrect after withdrawal with sufficient balance.")
assert(acct.withdraw(100) == 50, "Withdrawal returned wrong amount when balance isn't sufficient.")
assert(acct.balance == 0, s"After attempted overdraft, account balance should be 0, but is ${acct.balance}")

### Object-private fields

In Scala, as in Java and other languages, a method can access the private fields of all objects of its class:

In [ ]:
class Foo {
  private var myValue = 0
  
  def examineOther(o: Foo) = o.myValue
  
  def value = myValue
  
  def value_=(newValue: Int): Unit = {
    myValue = newValue
  }
}

val first = new Foo
first.value = 100
val second = new Foo

In [ ]:
println(s"second says first's value is ${foo2.examineOther(foo1)}")

- Scala allows you to make a private field visible _only_ within an instance. 
- Use the `private[this]` access modifier. 

With that change, the following code no longer compiles.

In [ ]:
class Foo {
  private[this] var myValue = 0
  
  def examineOther(o: Foo) = o.myValue
  
  def value = myValue
  
  def value_=(newValue: Int): Unit = {
    myValue = newValue
  }
}


Another side-effect of creating a `private[this]` field is that, unlike a `private` field, Scala will _not_ generate getters or setters for the field.

Why do we care about this? Because:

- It's kind of cool.
- It's one instance of Scala's far richer access semantics.
- As we'll see shortly, `private[this]` can come into play when we define class constructors.

### Bean properties

Some third-party libraries you use expect objects you pass to them to adhere to the [JavaBeans Specification](http://www.oracle.com/technetwork/java/javase/documentation/spec-136004.html).
As this [StackOverflow answer](http://stackoverflow.com/a/3295517/53495) notes, a JavaBean must:

- have _private_ properties
- have Java-style getters and setters (e.g., property `x` must be set and accessed via `setX()` and `getX()` methods)
- have a public no-argument constructor
- implement `Serializable`

But Scala classes do not adhere to that convention. If you need to pass an instance of a Scala object to a library that wants a JavaBean-compliant
object, what do you do? Do you have to implement all those (annoying) `set` and `get` methods yourself?

No. You don't.

Instead, you can mark a variable with the `BeanProperty` annotation. For instance:

In [ ]:
import scala.beans.BeanProperty

class Person {
  @BeanProperty var age = 0
  @BeanProperty var name = ""
}

val p = new Person
p.age = 10
assert(p.age == 10)
assert(p.age == p.getAge)
p.setAge(20)
assert(p.age == 20)


Using `@BeanProperty` on the `var` field `name` caused Scala to generate _four_ methods, instead of the usual two:

1. `name`: Scala getter (get the name as a string)
2. `name_=`: Scala setter
3. `getName`: JavaBean getter
4. `setName`: JavaBean setter

If you use `@BeanProperty` on a `val` field, it only generates the Scala getter and the Java getter.

Using `@BeanProperty`, you can create classes that can be used seamlessly with Java libraries expecting JavaBeans.


### Constructors

Our `Person` class is a bit clunky. Instead of passing values into the class to initialize it, we have to create an
empty instance and then call setters. Let's clean it up.

#### The primary constructor

In most languages, the primary constructor defined as a special method inside the class. For instance, in Java, our `Person` class's primary constructor might look like this:

```java
public class Person {
    private String name = "";
    private int age = 0;
    
    public Person(String name, int age) {
        this.name = name;
        this.age = age;
    }
    
    // Getters and setters go here
}
```

In Scala, the primary constructor syntax is moth more compact _and_ more readable, yet still readable. 

- The parameters of the primary constructor go right after the class name. 

In this example, we'll make the `Person` class immutable:

In [ ]:
class Person(val name: String, val age: Int)

val p = new Person("Fred", 38)

In [ ]:
println(s"${p.name} is ${p.age}")

**A complete class, declared in a single line of code!**

Parameters of the primary constructor turn into fields that are initialized with the construction parameters.

Compare our 1-line class definition with the equivalent Java class definition:

```scala
// Scala
class Person(val name: String, val age: Int)
```

```java
// Java
public class Person {
    private String name = "";
    private int age = 0;
    
    public Person(String name, int age) {
        this.name = name;
        this.age = age;
    }
    
    public String getName() {
        return this.name;
    }
    
    public int getAge() {
        return this.age;
    }
} 
``` 


#### Code in the primary constructor

What if our primary constructor needs some actual logic? 

- In Java, you put it in the primary constructor method.
- In Scala, that code just goes inline, in the class.


In [ ]:
class Person(val firstName: String, val lastName: String, val age: Int) {
  val fullName = s"$firstName $lastName" // This assignment runs as part of the primary constructor code.
}

val p = new Person("Melinda", "McPherson", 47)

In [ ]:
println(s"${p.fullName} is ${p.age}")

**A little more formally:** The primary constructor executes _all_ statements in the class definition.

This is also valid:

In [ ]:
class Bar(val value: Int) {
  val derivedValue = value * 2
  
  def somethingRandom = value + scala.util.Random.nextInt
  
  println(s"value is $value, and derived value is $derivedValue")
  
}

In [ ]:
val b = new Bar(10)
println("----- Constructor is done.")
println(b.somethingRandom)
println(b.somethingRandom)

#### Auxiliary Constructors

- As in Java, a Scala class can have as many constructors as you want. In addition to the primary constructor, you can define _auxiliary_ constructors.
- Unlike Java, though, these auxiliary instructors are _not_ named after the class. They're just called `this`. 
- An auxiliary instructor _must_ chain to another auxiliary instructor or to the primary constructor.


In [ ]:
// Primary constructor requires a name and an age.
class Person(val name: String, val age: Int) {
  
  // This auxiliary constructor takes only a name and uses a default value for the age.
  def this(name: String) = this(name, 0)
  
  // This auxiliary constructor takes only an age and uses a default value for the name.
  def this(age: Int) = this("?", age)
}

val p1 = new Person("Maria")
println(s"${p1.name} is ${p1.age} years old.")

val p2 = new Person(10)
println(s"${p2.name} is ${p2.age} years old.")


**But you might not need auxiliary constructors**

You can often get the same behavior with default parameter values. 

In [ ]:
// Primary constructor requires a name and an age.
class Person(val name: String = "?", val age: Int = 0)

val p1 = new Person("Maria")
println(s"${p1.name} is ${p1.age} years old.")

val p2 = new Person(age=10)
println(s"${p2.name} is ${p2.age} years old.")

#### What about constructor parameters that aren't `val` or `var`?

What if you define a class, but you don't put `val` or `var` in front of the constructor parameters?

One of two things happens:

* If the parameter is _only_ used in the constructor logic, then it's just a parameter. It lives on the constructor method's stack,
  and it disappears after the constructor returns.
* If the parameter is referenced in _any_ method, then it can't be on the stack. It has to hang around for the lifetime of the object.
  In that case, Scala stores its value in a `private[this]` field.

### Exercise 5.2: Use your understanding of constructors to fill in the following class

**Requirements**

The `Employee` class must have these properties:

* A read-only `firstName` field.
* A read-only `lastName` field.
* A `birthDateString` parameter, which will be a string of the form `yyyy-mm-dd` (e.g., `"1990-10-30"`). Don't worry about
  invalid date strings. Use the `parseDate` method, supplied, to parse it.
* A read-only (derived) `birthDate` field, of type `java.util.Date`.
* A read-only (derived) `age` field.
* A read-only (derived) `fullName` property that puts the first and last name together.
* An optional `ssn` (Social Security Number) field, of type `Option[String]`.

Support these constructor patterns:

* `new Employee("Melody", "Smythe", "1983-10-03", Some(966-40-9084))` — all parameters specified.
* `new Employee("Ravi", "Kumar", "1956-01-17)` — no SSN specified

The class stub contains some private helper methods you might find useful.

Use the cell following your exercise cell to test your solution.

In [ ]:
import java.time.LocalDate

class Employee(/* fill this in */) {
  /* fill this in */
  
  private def calcAge(birthDate: LocalDate): Int = {
    import java.time.Period
    Period.between(d, LocalDate.now).getYears
  }
  
  private def parseDate(dateString: String): LocalDate = {
    import java.time.format.DateTimeFormatter
    import java.time.LocalDate
    LocalDate.parse(dateString, DateTimeFormatter.ISO_LOCAL_DATE)
  }
}

In [ ]:
// ANSWER
import java.time.LocalDate

class Employee(val firstName:       String,
               val lastName:        String,
               val birthDateString: String,
               val ssn:             Option[String] = None) {
  val birthDate = parseDate(birthDateString)
  val age = calcAge(birthDate)
  val fullName = s"$firstName $lastName"
  
  private def calcAge(birthDate: LocalDate): Int = {
    import java.time.Period
    Period.between(d, LocalDate.now).getYears
  }
  
  private def parseDate(dateString: String): LocalDate = {
    import java.time.format.DateTimeFormatter
    import java.time.LocalDate
    LocalDate.parse(dateString, DateTimeFormatter.ISO_LOCAL_DATE)
  }
}

In [ ]:
// Run this cell to test your solution
val e1 = new Employee("John", "Smallberries", "1959-10-31", Some("923-37-5027"))
val e2 = new Employee("Anisa", "Gutman", "1989-02-16")

assert(e1.fullName == "John Smallberries", "e1.fullName isn't correct")
assert(e1.lastName == "Smallberries", "e1.lastName isn't correct")
assert(e1.ssn.getOrElse("") == "923-37-5027", "e1.ssn isn't correct")

assert(e1.birthDate.getYear == 1959, "e1.birthDate has the wrong year")
assert(e1.birthDate.getMonthValue == 10, "e1.birthDate has the wrong month")
assert(e1.birthDate.getDayOfMonth == 31, "e1.birthDate has the wrong day")

assert(e2.fullName == "Anisa Gutman", "e1.fullName isn't correct")
assert(e2.firstName == "Anisa", "e2.firstName isn't correct")
assert(e2.ssn.isEmpty, "e2.ssn is defined, but shouldn't be")

assert(e2.birthDate.getYear == 1989, "e2.birthDate has the wrong year")
assert(e2.birthDate.getMonthValue == 2, "e2.birthDate has the wrong month")
assert(e2.birthDate.getDayOfMonth == 16, "e2.birthDate has the wrong day")

### Basic Inheritance

You can inherit from a class, as you would expect. Just like Java, use the `extends` keyword:

In [ ]:
class Foo {
  val i: Int = scala.math.abs(scala.util.Random.nextInt)
}

class Bar extends Foo

val foo = new Foo
val bar = new Bar

println(foo.i)
println(bar.i)

## Chaining superclass parameters

In [ ]:
class Customer(name: String, age: Int, val customerID: String) extends Person(name, age)

val customer = new Customer("John Bigbooté", 55, "X0982349-AL")

println(s"${customer.name}, ${customer.age}, ${customer.customerID}")

If you don't want anyone to be able to subclass your class, mark it `final`.

In [ ]:
final class Quux {
  var i: Int = 0
}

In [ ]:
// Won't compile
class Fribble extends Quux

### Abstract classes

Scala, like Java, supports abstract classes. An abstract class cannot be instantiated, but it's a good place to stash common logic.

* Abstract classes are marked with the keyword `abstract`. 
* They can also have abstract methods (methods without a body) and abstract fields (field definitions without an initializer).
* Their constructors can take parameters.

In [ ]:
abstract class Pet(val age: Int) {
  val name: String // no initializer => abstract field
  
  def speak: String // no body => abstract
}

Any subclass of `Pet` _must_ provide a `name` field and a `speak` method, or it will not compile:

In [ ]:
// Won't compile
class BadDog(age: Int, val name: String) extends Pet(age)

In [ ]:
class Dog(age: Int, val name: String) extends Pet(age) {
  def speak = "Woof!"
}

#### You can override a `def` with a `val`

In [ ]:
class RefinedDog(age: Int, val name: String) extends Pet(age) {
  val speak = "Woof!"
}

The rules for overriding fields and methods:

* A `def` in a subclass can _only_ override a `def` in a superclass. (In other words, a `def` cannot override a `val` or a `var`.)
* A `val` in a subclass can only override another `val` or a parameterless `def`.
* A `var` can only override an abstract `var`.

![!](http://i.imgur.com/TvgcpDH.png) **Don't use a public `var` in a non-final class!**

- While you can always hide it later, with your own custom getter and setter, anyone _extending_ your class doesn't have that option.
- They _cannot_ override a `var` with a getter/setter pair.
- **If you provide a `var` in a superclass, all subclasses are stuck with it.**

In [ ]:
// Won't compile
class MyBaseClass {
  var i: Int = 0
}

class MySubClass extends MyBaseClass {
  var _i: Int = 0
  override def i = _i
  override def i_=(newI: Int): Unit = {_i = newI}
}

### Overriding concrete members vs overriding abstract members

When overriding a concrete member in a superclass, you _must_ use the `override` keyword. 

`override` is like Java's `@Override` annotation, except:

- It's a keyword.
- It's not optional.

When overriding an abstract member, you do not need `override`.

Examples:

In [ ]:
abstract class SomeBase {
  def abstractMethod1: String
  
  def concreteMethod1: String = "foo"
}

In [ ]:
// Will compile

class GoodSubclass extends SomeBase {
  val abstractMethod1 = "Rickrollin'..."
  override val concreteMethod1 = "...sucks"
}

In [ ]:
// Won't compile

class BadSubclass extends SomeBase {
  val abstractMethod1 = "Can we take a break now?"
  val concreteMethod1 = "I could use some caffeine."
}

## Traits

Conceptually, a trait is similar to a [Java 8 default method implementation](https://docs.oracle.com/javase/tutorial/java/IandI/defaultmethods.html) in a Java interface. However, traits are considerably more powerful.

Like Java, Scala only permits you to extend one base class. Also like Java, you can mix in multiple traits (interfaces). Traits can also impose requirements on classes that mix them in.

### Summary

* A class can implement any number of traits.
* Traits can require that implementing classes have certain fields, methods, or superclasses.
* A trait can provide default implementations for methods and fields.
* When you layer multiple traits, order matters: The trait whose methods execute _first_ is the last one in the list.

### Traits as interfaces

A Scala trait can work exactly like an interface.

In [ ]:
trait CanLog {
  def log(msg: String): Unit  // abstract method
}

A subclass can now provide an implementation.

**NOTE**: When implementing a single trait, using `extends`, just as you would when extending a class.

In [ ]:
class ConsoleLogger extends CanLog {
  def log(msg: String): Unit = {
    println(msg)
  }
}

val logger = new ConsoleLogger
logger.log("One of the flayrods has gone out on treadle.")

#### Extending multiple traits

If you need more than one trait, add the others using the `with` keyword. Note that Scala treats Java interfaces as if they were traits.

In [ ]:
class ConsoleLogger extends CanLog with Cloneable with Serializable {
  def log(msg: String): Unit = {
    println(msg)
  }
}

![?](http://i.imgur.com/Guv4TBn.png) **Why do we use `extends` on the first one, but `with` for all the others?**

Scala actually views the inheritance like this:

```
ConsoleLogger extends [CanLog with Cloneable with Serializable]
```

In other words, `ConsoleLogger` is extending a _composite_ that consists of `CanLog` with `Cloneable` and `Serializable`.

### Traits with implementations

A trait's methods need not be abstract.

In [ ]:
trait ConsoleLogger extends CanLog {
  def log(msg: String): Unit = println(msg)
}

We now have a trait with a method, and since it's a trait, it can be mixed into other classes.

In [ ]:
class Customer(name: String, age: Int, val customerID: String) extends Person(name, age) with ConsoleLogger {
  def recordPurchase(product: String, quantity: Int, price: BigDecimal): Unit = {
    log(s"""Customer $customerID just bought "$product" (quantity $quantity) for $price""")
  }
}

val customer = new Customer("John Yaya", 47, "Y12994310-AL")
customer.recordPurchase("overthruster", 1, 957.99)

![!](http://i.imgur.com/TvgcpDH.png) There's one problem with traits that have concrete implementions: When such a trait changes, _all_ class that mix the trait in _must_ be recompiled.

### Fields in traits

Traits can have fields (values), not just methods.

#### Concrete fields

Traits can have concrete (non-abstract) fields. Consider:

In [ ]:
trait ShortLogger extends CanLog {
  val maxLength = 15
  
  abstract override def log(msg: String): Unit = {
    super.log(
      if (msg.length < maxLength) msg else msg.take(maxLength - 3).mkString("") + "..."
    )
  }
}

Any class that mixes `ShortLogger` in acquires a `maxLength` field. The field is added _directly_ to (i.e, mixed into) the class; it's not inherited. 

In [ ]:
val log = new ConsoleLogger with ShortLogger
log.log("This is a message that exceeds 15 characters. It should be truncated before being written to the console.")

You can think of concrete trait fields as "assembly instructions" for the classes that use the trait. Any such fields become fields of the class.

<div style="border: 1px solid #ddd; border-radius: 10px 10px 10px 10px; padding: 10px; background-color: #f7edcd; width: 40%; margin: 20px">
<h3>Why do I need <tt>abstract override</tt> when <tt>CanLog.log()</tt> is already abstract?</h3>

<p>Scala has no way of knowing which <tt>log()</tt> method <tt>super.log()</tt> is invoking, because it all depends on what traits
are mixed in and in what order. So, the Scala compiler takes the position that <tt>log()</tt> is still abstract, even though we're
providing a default implementation. It still requires that some <i>concrete</i> implementation be mixed in. Thus, you have to define
the <tt>ShortLogger</tt> version of <tt>log()</tt> with <tt>abstract override</tt>.</p>
</div>

#### Abstract fields

Traits can also have abstract fields. Any concrete class that mixes in the trait must supply an concrete value for the field. Let's demonstrate with a variation of the `ShortLogger` trait that doesn't hard-code the maxLength.

In [ ]:
trait ShortLogger extends CanLog {
  val maxLength: Int  // no initialized value, so abstract
  
  override abstract def log(msg: String): Unit = {
    super.log(
      if (msg.length < maxLength) msg else msg.take(maxLength - 3).mkString("") + "..."
    )
  }
}

Now, any attempt to instantiate a class that mixes in `ShortLogger` will fail, unless we also provide a `maxLength` concrete field.

In [ ]:
// This fails
val log = new ConsoleLogger with ShortLogger {
  override def log(msg: String): Unit = super.log(msg)
}

In [ ]:
// This succeeds
val log = new ConsoleLogger with ShortLogger {
  val maxLength = 20
  
  override def log(msg: String): Unit = super.log(msg)
}

In [ ]:
log.log("This is a message that exceeds 20 characters. It should be truncated before being written to the console.")

### Trait construction order and layered traits

Just like classes, traits can have primary constructors. For instance:

In [ ]:
trait FileLogger extends CanLog {
  import java.io.{PrintWriter, FileOutputStream}
  val out = new PrintWriter(new FileOutputStream("/tmp/phase.log", true))
  out.println(s"# STARTING NEW LOG AT ${new java.util.Date}")
  
  abstract override def log(msg: String): Unit = {
    out.println(msg)
    out.flush()
    super.log(msg)
  }
}

In [ ]:
val log = new ConsoleLogger with FileLogger {
  override def log(msg: String): Unit = super.log(msg)
}

log.log("This is a message that should go to the file and the console.")

In [ ]:
// What's in the file?
import scala.sys.process._

println("cat /tmp/phase.log".!!)

With traits mixed in, here's how object construction works:

1. The superclass constructor is called first.
2. Trait constructors are executed next, _before_ the class constructor.
3. Traits are constructed left to right, but:
4. ...within each trait, _parents_ are constructed first.
5. If multiple traits share a common parent, and that parent has been constructed already, it is not constructed again.
6. After all traits are constructed, the subclass is constructed.

Confused? Let's clarify with an example.

In [ ]:
abstract class Account(account: String) extends CanLog {
  val accountNumber = account.replace("-", "")
  
  override def log(msg: String): Unit = {}
}

class SavingsAccount(accountNumber: String, val interestRate: BigDecimal) extends Account(accountNumber) with FileLogger with ShortLogger {
  val maxLength = 50
}

In [ ]:
val acct = new SavingsAccount("203-94-87234-1", BigDecimal(0.3))
acct.log(s"Acct number: ${acct.accountNumber}")

In [ ]:
println("cat /tmp/phase.log".!!)

The `SavingsAccount` object is constructed in this order:

1. The `Account` constructor fires (initializing `accountNumber`).
2. The `CanLog` trait is constructed, as it's the parent of the first trait, `FileLogger`.
3. `FileLogger` is constructed next.
4. Then, `ShortLogger` is constructed—but, since its parent, `CanLog` has already been constructed, it isn't constructed again.
5. Finally, `SavingsAccount` is constructed.

This approach lends itself nicely to using traits as an on-the-fly composition mechanism. To save time, instead
of doing another exercise, let's just look at a couple quick examples.

In [ ]:
// Logger that adds a timestamp to a log message and forwards the result along.
trait TimestampLogger extends CanLog {
  import java.time.{ZonedDateTime, ZoneId}
  import java.time.format.DateTimeFormatter
  
  private val format = DateTimeFormatter.ofPattern("yyyy/mm/dd HH:MM:ss.SSS")
  abstract override def log(msg: String): Unit = {
    // Always log in UTC
    val now = ZonedDateTime.now(ZoneId.of("UTC"))

    super.log(s"${format.format(now)}: $msg")
  }
}

In [ ]:
val log1 = new ConsoleLogger with ShortLogger with TimestampLogger {
  val maxLength = 24
  override def log(msg: String): Unit = super.log(msg)
}
val log2 = new ConsoleLogger with TimestampLogger with ShortLogger {
  val maxLength = 24
  override def log(msg: String): Unit = super.log(msg)
}

Okay, watch carefully. One of you will be asked to explain what's going on here.

In [ ]:
val msg = "Strange women lying in ponds is no basis for a system of government."

log1.log(msg)
log2.log(msg)

## Objects

Scala supports the notion of a singleton object, and that's the final concept we'll be covering in this lesson.

### Summary

* Objects are for singletons and utility methods.
* A class can have a companion object of the same name.
* Objects can extend classes and traits.
* The `apply()` method of an object is special.

### Where the \*@$! is the `static` keyword?

Scala has no `static` keyword. Classes must be instantiated to call any methods on them.

Instead of `static` methods and `static` classes, Scala has singleton objects. For example:

In [ ]:
object Stat extends ConsoleLogger {
  def median(items: Double*): Double = {
    val itemList = items.toList
    val len = itemList.length
    require (len > 0)

    if (len == 1)
      itemList.head

    else {
      val sorted = itemList sortWith(_ < _)
      val midpoint = sorted.length / 2
      if (len % 2 == 0)
        mean(sorted(midpoint), sorted(midpoint - 1))
      else
        sorted(midpoint)
    }
  }
  
  def mean(items: Double*): Double = {
    val itemList = items.toList
    val len = itemList.length
    require (len > 0)

    if (len == 1)
      itemList.head
    else
      itemList.foldLeft(0.0)((a, b) => a + b) / len
  }
}

In [ ]:
val values = (1 to 40).map { _ => scala.util.Random.nextInt(20).toDouble }
println(s"Values = ${values.mkString(", ")}")
println(s"mean=${Stat.mean(values: _*)}, median=${Stat.median(values: _*)}")
println("-" * 30)

In [ ]:
%sh

### Objects as packages

As noted in a previous unit, you can import from objects. This feature means you can
also treat singleton objects as packages.

In [ ]:
object org {
  object example {
    object stat {
      def mean(items: Double*): Double = {
        val itemList = items.toList
        val len = itemList.length
        require (len > 0)

        if (len == 1)
          itemList.head
        else
          itemList.foldLeft(0.0)((a, b) => a + b) / len
      }
    }
  }
}

In [ ]:
import org.example.stat.mean

mean(10, 20, 5, 14, 13.3)

### Companion objects

A common pattern in Java is the factory pattern, where you construct an object by calling a static method rather than the constructor. Without static methods, how do we do the same thing in Scala?

In Scala, you can have a `class` and an `object` that have the same name. As long as they're defined in the same source file, they're called _companion objects_.

In [ ]:
class Account(val accountNumber: String)

object Account {
  def create(accountNumber: String) = new Account(accountNumber)
}

Now, I can create an `Account` object two ways:

In [ ]:
val a1 = new Account("0927834")
val a2 = Account.create("87654233")

#### Companions are special

But there's more to it than that. Companion objects share a special relationship: They can (you'll pardon the expression) _see each other's privates_.

Let's make the `Account` constructor private, and try again.

In [ ]:
class Account private(val accountNumber: String)

object Account {
  def create(accountNumber: String) = new Account(accountNumber)
}

With that change, I can no longer call the constructor. I have no choice but to use
the `create` function.

In [ ]:
val a1 = new Account("0927834")

In [ ]:
val a1 = Account.create("0927834")

### The `apply()` method

There's a special method you can use on _any_ object (whether instantiated from a class or an `object` singleton) called `apply()`. Scala has special syntactic sugar for `apply`, best described by example. Let's convert our `Account` example so that `create` is called `apply`.

In [ ]:
class Account private(val accountNumber: String)

object Account {
  def apply(accountNumber: String) = new Account(accountNumber)
}

Naturally, I can now create an account by calling `Account.apply()`. But, Scala has a shorter syntax. If I try to invoke the object as if it were a function, like this:

```scala
Account("871324")
```

the compiler turns that into a call to `apply()`:

```scala
Account.apply("871324")
```

In [ ]:
// These are equivalent
val a1 = Account("871324")
val a2 = Account.apply("871324")

println(a1.accountNumber)
println(a2.accountNumber)

#### Aside

This is why Scala's array subscript "operator" is parentheses (`()`), not square brackets (`[]`): There's an `apply()` method on the `Array` _class_. You can call the method on any Array _instance_ to subscript the array.

In [ ]:
val arr = (1 to 30 by 2).toArray

println(arr(0))
println(arr.apply(0))

### Objects can extend traits and classes

A singleton `object` can extend any class or trait.

In [ ]:
object MyLogger extends ConsoleLogger with TimestampLogger with ShortLogger {
  val maxLength = 80
}

In [ ]:
MyLogger.log("Your mother called. She says you never call her.")

### Singleton objects do _not_ have constructor parameters

- You can put logic in the object's constructor, exactly as you would do with a class or trait. 
- _But_ objects cannot be passed constructor parameters. 

An `object` is a singleton. If you want objects that can be instantiated with different parameters, use a `class`.

In [ ]:
object Foo {
  println("This will fire when I first use the object.")
  
  def doSomethingWith(i: Int) = println(i)
}

In [ ]:
Foo.doSomethingWith(15)

In [ ]:
Foo.doSomethingWith(20)

### Objects as main programs

There are two ways to build a main program in Scala. The most straightforward way is to write an object that contains a `main()` method:

In [ ]:
object MyProgram1 {
  def main(args: Array[String]): Unit = {
    println("Bonjour, tout le monde.")
    args.foreach(println)
  }
}

We can't actually run the program here. But, assuming you saved that code to a file
called `MyProgram1.scala`, and you've installed Scala on your laptop, you can compile
and run it with these commands:

```shell
$ scalac -d . MyProgram1.scala
$ scala -cp . MyProgram1
```

Another way to create a main program in Scala is to use the special `App` trait. Then, everything in the `object` constructor becomes the main program, and the arguments are available as `args`:

In [ ]:
object MyProgram2 extends App {
  println("Hola, mundo.")
  args.foreach(println)
}

## That's all for Classes, Traits and Objects!